In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

import sys
sys.path.append('../src/')
from models.evaluation import get_classification_report 

In [2]:
RESULTS_PATH = "../results/"
IMAGES_PATH = "../images/"

In [3]:
with open(RESULTS_PATH + "history/fcnn_history.json", 'r') as file:
    
    fcnn_history = json.loads(file.read())
    
with open(RESULTS_PATH + "history/fcnn_aug_history.json", 'r') as file:
    
    fcnn_aug_history = json.loads(file.read())
    
with open(RESULTS_PATH + "history/cnn_history.json", 'r') as file:
    
    cnn_history = json.loads(file.read())
    
with open(RESULTS_PATH + "history/cnn_aug_history.json", 'r') as file:
    
    cnn_aug_history = json.loads(file.read())
    
with open(RESULTS_PATH + "history/cnn_without_polling_history.json", 'r') as file:
    
    cnn_wo_poll_history = json.loads(file.read())
    
with open(RESULTS_PATH + "history/benchmark_cnn_history.json", 'r') as file:
    
    benchmark_cnn_history = json.loads(file.read())


In [4]:
test_results_benchmark = pd.read_csv(RESULTS_PATH + 'test/cnn_benchmark_test_results.csv')
train_results_benchmark = pd.read_csv(RESULTS_PATH + 'train/cnn_benchmark_train_results.csv')

train_results_cnn = pd.read_csv(RESULTS_PATH + 'train/cnn_train_results.csv')
test_results_cnn = pd.read_csv(RESULTS_PATH + 'test/cnn_test_results.csv')

train_results_cnn_wo_poll = pd.read_csv(RESULTS_PATH + 'train/cnn_without_polling_train_results.csv')
test_results_cnn_wo_poll = pd.read_csv(RESULTS_PATH + 'test/cnn_without_polling_test_results.csv')

train_results_cnn_aug = pd.read_csv(RESULTS_PATH + 'train/cnn_aug_train_results.csv')
test_results_cnn_aug = pd.read_csv(RESULTS_PATH + 'test/cnn_aug_test_results.csv')

train_results_fcnn = pd.read_csv(RESULTS_PATH + 'train/fcnn_train_results.csv')
test_results_fcnn = pd.read_csv(RESULTS_PATH + 'test/fcnn_test_results.csv')

train_results_fcnn_aug = pd.read_csv(RESULTS_PATH + 'train/fcnn_aug_train_results.csv')
test_results_fcnn_aug = pd.read_csv(RESULTS_PATH + 'test/fcnn_aug_test_results.csv')

## ANALYSIS

In [5]:

def model_report(
    name, 
    history,
    train_results,
    test_results
    
):
    
    # summarize history for accuracy
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.title(f'Acurácia do modelo ao longo das épocas - {name}')
    plt.ylabel('acurácia')
    plt.xlabel('época')
    plt.legend(['treino', 'validação'], loc='upper left')
    plt.savefig(IMAGES_PATH + f'results_plt/{name.lower().replace(" ", "_")}_accuracy_progress.png')

    plt.clf()

    # summarize history for loss
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title(f'Loss do modelo ao longo das épocas - {name}')
    plt.ylabel('loss')
    plt.xlabel('época')
    plt.legend(['treino', 'validação'], loc='upper left')
    plt.savefig(IMAGES_PATH + f'results_plt/{name.lower().replace(" ", "_")}_loss_progress.png')
        
    plt.clf()   
    
    # summarize history for loss
    plt.plot(history['learning_rate'])
    plt.title(f'Taxa de aprendizado ao longo das épocas - {name}')
    plt.ylabel('Taxa de aprendizado')
    plt.xlabel('época')
    plt.legend(['treino', 'validação'], loc='upper left')
    plt.savefig(IMAGES_PATH + f'results_plt/{name.lower().replace(" ", "_")}_learning_rate_progress.png')
    
    plt.clf()
    
    df_report_test = get_classification_report(test_results['y_test'], test_results['y_test_pred']).drop('support',axis= 1)
    df_report_train = get_classification_report(train_results ['y_train'], train_results['y_train_pred']).drop('support',axis= 1)
    
    df_report_test.reset_index().rename({'index': 'class'},axis=1).to_csv(
        RESULTS_PATH + f'metrics/{name.lower().replace(" ", "_")}_test_results.csv',
        index=False
        )
    df_report_train.reset_index().rename({'index': 'class'},axis=1).to_csv(
        RESULTS_PATH + f'metrics/{name.lower().replace(" ", "_")}_train_results.csv',
        index=False
        )
    df_report_test.to_markdown(
        RESULTS_PATH + f'metrics_md/{name.lower().replace(" ", "_")}_test_results.md'
        )
    df_report_train.to_markdown(
        RESULTS_PATH + f'metrics_md/{name.lower().replace(" ", "_")}_train_results.md'
        )
    
    df_report_test.to_html(
        RESULTS_PATH + f'metrics_html/{name.lower().replace(" ", "_")}_test_results.html'
        )
    df_report_train.to_html(
        RESULTS_PATH + f'metrics_html/{name.lower().replace(" ", "_")}_train_results.html'
        )
    df_report_test.to_latex(
        RESULTS_PATH + f'metrics_latex/{name.lower().replace(" ", "_")}_test_results.tex'
        )
    df_report_train.to_latex(
        RESULTS_PATH + f'metrics_latex/{name.lower().replace(" ", "_")}_train_results.tex'
        )

In [6]:
model_report(
    name = 'FCNN',
    history = fcnn_history,
    train_results = train_results_fcnn,
    test_results = test_results_fcnn  
)

model_report(
    name = 'CNN',
    history = cnn_history,
    train_results = train_results_cnn,
    test_results = test_results_cnn  
)

model_report(
    name = 'CNN sem polling', 
    history = cnn_wo_poll_history,
    train_results = train_results_cnn_wo_poll,
    test_results = test_results_cnn_wo_poll  
)

model_report(
    name = 'CNN com data augmentation', 
    history = cnn_aug_history,
    train_results = train_results_cnn_aug,
    test_results = test_results_cnn_aug
)

model_report(
    name = 'FCNN com data augmentation', 
    history = fcnn_aug_history,
    train_results = train_results_fcnn_aug,
    test_results = test_results_fcnn_aug
)

<Figure size 640x480 with 0 Axes>

In [7]:
df_f1_score = pd.DataFrame({})

for i,name in enumerate(['CNN',  'CNN com data augmentation', 'CNN sem polling','FCNN','FCNN com data augmentation']):
    
    
    df = pd.read_csv( RESULTS_PATH + f'metrics/{name.lower().replace(" ", "_")}_test_results.csv')
    
    df = df.drop(['recall','precision'], axis=1)
    
    df.rename({'f1-score': name},axis=1, inplace=True)
    
    
    if i == 0: 
        df_f1_score = df
    else:
        df_f1_score = df_f1_score.merge(
            df,
            on = 'class'
        )


In [8]:
df_f1_score.rename({'class':'classe'},axis=1,inplace=True)

In [9]:
df_f1_score = df_f1_score[~df_f1_score.classe.isin(['weighted avg','accuracy'])]
df_f1_score

,classe,CNN,CNN com data augmentation,CNN sem polling,FCNN,FCNN com data augmentation
0,0,0.911392,0.950787,0.916834,0.912670,0.951437
1,1,0.872821,0.904192,0.886725,0.877778,0.910660
2,2,0.848276,0.885028,0.835381,0.840909,0.867603
3,3,0.922330,0.925908,0.920174,0.926017,0.918715
4,4,0.868132,0.903421,0.873689,0.873924,0.911201
5,5,0.901305,0.926316,0.897622,0.907398,0.923239
6,6,0.876490,0.927159,0.890602,0.888994,0.899572
7,7,0.903796,0.944444,0.902490,0.900818,0.947098
8,8,0.869398,0.917044,0.877295,0.876339,0.921066
9,9,0.900152,0.938816,0.901597,0.899384,0.940939


In [10]:
print(df_f1_score.to_latex(float_format="%.4f", index=False))

\begin{tabular}{lrrrrr}
\toprule
classe & CNN & CNN com data augmentation & CNN sem polling & FCNN & FCNN com data augmentation \\
\midrule
0 & 0.9114 & 0.9508 & 0.9168 & 0.9127 & 0.9514 \\
1 & 0.8728 & 0.9042 & 0.8867 & 0.8778 & 0.9107 \\
2 & 0.8483 & 0.8850 & 0.8354 & 0.8409 & 0.8676 \\
3 & 0.9223 & 0.9259 & 0.9202 & 0.9260 & 0.9187 \\
4 & 0.8681 & 0.9034 & 0.8737 & 0.8739 & 0.9112 \\
5 & 0.9013 & 0.9263 & 0.8976 & 0.9074 & 0.9232 \\
6 & 0.8765 & 0.9272 & 0.8906 & 0.8890 & 0.8996 \\
7 & 0.9038 & 0.9444 & 0.9025 & 0.9008 & 0.9471 \\
8 & 0.8694 & 0.9170 & 0.8773 & 0.8763 & 0.9211 \\
9 & 0.9002 & 0.9388 & 0.9016 & 0.8994 & 0.9409 \\
macro avg & 0.8874 & 0.9223 & 0.8902 & 0.8904 & 0.9192 \\
\bottomrule
\end{tabular}

